# OpenMM Tutorial

Platform for molecular dynamics. Customizable, open-source, supports CUDA, etc.

OpenMM has weird documentation. Often the first link if you google openmm points you to an old version (7.0.0). Look for the 8.1.0 version if you are up to date.

## Imports

Along with the main openmm import, there are two libraries that are particularly useful: openmm.app and openmm.unit. openmm.unit is appropriately named and allows users to define simulations in the unit system they prefer.

openmm.app is a set of libraries for the high-level application layer. This provides easier interfacing with your code to tune general parameters and run simulations. We will focus on this to begin, and we will return to the lower-level libraries in openmm later in the tutorial.

In [1]:
import openmm as mm
import openmm.app as app
import openmm.unit as u
import numpy as np

## Input structures and forcefields

OpenMM supports multiple types of input files. The most familiar and useful for us is the PDB filetype, though .gro (Gromacs), .inpcrd/.prmtop (AMBER), and .psf (CHARMM) are also used.$^*$ A PDB file is read using the application layer, and the topology and coordinates are stored in a PDBFile object. 


###### $^*$ These usually require both coordinate and topology files, and the system is created on the topology object. 

In [2]:
pdb = app.PDBFile("alanine-dipeptide.pdb")
print(pdb.topology)
print(pdb.positions)

<Topology; 1 chains, 3 residues, 22 atoms, 21 bonds>
[Vec3(x=0.2, y=0.1, z=-0.0), Vec3(x=0.2, y=0.209, z=0.0), Vec3(x=0.1486, y=0.24540000000000003, z=0.08900000000000001), Vec3(x=0.1486, y=0.24540000000000003, z=-0.08900000000000001), Vec3(x=0.3427, y=0.2641, z=-0.0), Vec3(x=0.43910000000000005, y=0.1877, z=-0.0), Vec3(x=0.35550000000000004, y=0.397, z=-0.0), Vec3(x=0.27330000000000004, y=0.4556, z=-0.0), Vec3(x=0.4853, y=0.46140000000000003, z=-0.0), Vec3(x=0.5408000000000001, y=0.4316, z=0.08900000000000001), Vec3(x=0.5660999999999999, y=0.42210000000000003, z=-0.1232), Vec3(x=0.5123000000000001, y=0.4521, z=-0.21309999999999998), Vec3(x=0.663, y=0.47190000000000004, z=-0.1206), Vec3(x=0.5809000000000001, y=0.31410000000000005, z=-0.12410000000000002), Vec3(x=0.47130000000000005, y=0.6129, z=0.0), Vec3(x=0.36010000000000003, y=0.6653, z=0.0), Vec3(x=0.5846, y=0.6835, z=0.0), Vec3(x=0.6737000000000001, y=0.6359, z=-0.0), Vec3(x=0.5846, y=0.8284000000000001, z=0.0), Vec3(x=0.4819, y=0

To change your unit system, use the .value_in_unit() method. This gives the number in the new units, but it doesn't include the unit. Multiply by openmm.unit.(your_unit) to reintroduce it.

In [3]:
print(pdb.topology.getPeriodicBoxVectors()) # get box vectors in nm
print(pdb.topology.getPeriodicBoxVectors().value_in_unit(u.angstrom) * u.angstrom) # get box vectors in angstroms

(Vec3(x=10.0, y=0.0, z=0.0), Vec3(x=0.0, y=10.0, z=0.0), Vec3(x=0.0, y=0.0, z=10.0)) nm
(Vec3(x=100.0, y=0.0, z=0.0), Vec3(x=0.0, y=100.0, z=0.0), Vec3(x=0.0, y=0.0, z=100.0)) A


Forcefields are specified with their corresponding XML files and can be called with the ForceField class in the application layer. Many forcefields, such as amber14 shown below, are included with OpenMM and do not require the XML file where you are running the simulation.

In [4]:
forcefield = app.ForceField('amber14-all.xml')

Many times, water models are separated from the main forcefield and should be called separately (e.g. forcefield=app.ForceField('amber14-all.xml', 'amber14/tip3p.xml')). This lends flexibility in the choice of water model (implicit vs explicit, different methods, etc.). http://docs.openmm.org/latest/userguide/application/02_running_sims.html#force-fields provides an overview of a couple forcefields built into OpenMM. Note that some models require the topology to be adjusted to account for waters in the system.

You might sometimes want (or need) to use a forcefield that is not built into OpenMM. Most of the time, you can just download the appropriate xml file into the appropriate folder within the openmm package. There is also the OpenMMForceFields package itself (https://github.com/openmm/openmmforcefields) which expands the available range of forcefields available to you.

## System objects

The System defines what you are simulating using the forcefield, topology, nonbonded considerations, and other parameters depending on the forcefield you use. It is created by calling .createSystem() on your forcefield.

In [5]:
system = forcefield.createSystem(pdb.topology,
                                 nonbondedMethod=app.NoCutoff, # implies non-periodicity, isolated box in vacuum
                                                               # Use PME for periodic cases
                                 constraints=app.HBonds)

If simulating some bulk periodic system: to prevent issues with periodic images, the nonbondedCutoff must be less than $\frac{1}{2}({\rm box~vectors})$.

## Platform objects

On what hardware do you want to simulate your system? There are four options provided by OpenMM, but only two are worth considering for our uses: CUDA and CPU.$^*$ There are two options to implicitly choose a platform. (1) OpenMM defaults to the fastest option available, and (2) you can set the OPENMM_DEFAULT_PLATFORM environment variable to your choice.

Alternatively, to explicitly specify on which device you would like to run the calculations, simply do

###### $^*$ The other platforms are Reference and OpenCL. Reference is mainly a tool for writing your own platform and does not optimize performance. OpenCL works with GPUs and CPUs, is written in OpenCL, and is useful if you're working with non-Nvidia graphics cards.

In [6]:
platform = mm.Platform.getPlatformByName('CPU') # 'CUDA' if a GPU is available

## Integrators, thermostats, and barostats
For a list of available integrators, see http://docs.openmm.org/latest/userguide/application/02_running_sims.html#integrators.

#### NVE integrators, thermostats
E.g. VerletIntegrator

You can add a thermostat to these integrators to perform NVT simulations, but Langevin integrators are preferred. To add the thermostat, apply .addForce(your_thermostat) to the system object.

In [7]:
# way to remove forces (e.g. thermostat) from the system
def manual_remove_force(system, force_object):
    for i, force in enumerate(system.getForces()): # Checks all of the force contributions on the system
        if f'{force}'[0:50] == f'{force_object}'[0:50]: # Takes out any that match your argument
            system.removeForce(i)
            return True
    return False

In [8]:
nve_integrator = mm.VerletIntegrator(0.001*u.picoseconds)

thermostat = mm.AndersenThermostat(298.15*u.kelvin,
                                   1.0/u.picosecond)

manual_remove_force(system, thermostat) # Remove any existing thermostat (e.g. if the cell is re-run)

system.addForce(thermostat) # Add the thermostat
print(system.getForces()[-1])

<openmm.openmm.AndersenThermostat; proxy of <Swig Object of type 'OpenMM::AndersenThermostat *' at 0x7feb39dde1b0> >


#### NVT integrators
Langevin Middle Integrator: thermostat built into this, good for NVT simulations.

The app has a slightly different implementation called BAOAB Integrator - the difference here is that the Middle integrator returns half-step velocities while BAOAB returns on-step velocities. You might want one or the other depending on your use case.


#### NPT simulations
To run NPT simulations, you can add a barostat. Multiple kinds are built into OpenMM: isotropic, anisotropic, specifically for membranes, http://docs.openmm.org/development/api-python/library.html#forces. Note: the system must be periodic to use a barostat.

Like thermostats, barostats need to be added to the system as forces.

In [9]:
# remove thermostat if we use Langevin integrator
manual_remove_force(system, thermostat)
#print(system.getForces()) # check that the thermostat is removed

integrator = mm.LangevinMiddleIntegrator(300 * u.kelvin,           # temperature of the heat bath
                                         1.0 / u.picosecond,       # friction coefficient
                                         2.0 * u.femtosecond)      # integration timestep

barostat = mm.MonteCarloBarostat(1.0 * u.bar,       # pressure
                                 300 * u.kelvin,    # temperature - match with integrator
                                 25)                # frequency of volume change attempts in number of time steps

manual_remove_force(system, barostat) # prevent adding a second barostat if cell rerun

system.addForce(barostat)
print(system.getForces()[-1])

<openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x7feb39dd3510> >


Let's remove the barostat for now and just work with the NVT ensemble.

In [10]:
manual_remove_force(system, barostat)
print(system.getForces()) # check that the barostat is removed

[<openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7feb3e084a20> >, <openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x7feb3e084780> >, <openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x7feb3e0841b0> >, <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7feb39dd3120> >, <openmm.openmm.CMMotionRemover; proxy of <Swig Object of type 'OpenMM::CMMotionRemover *' at 0x7feb39dd3840> >]


#### Custom integrators
Instead of using an established integrator, you can also create your own. One example of this utility is with separated timescales; if you need to integrate over a fast process and a slow one, you can write an integrator that accomplishes this task efficiently:

In [11]:
custom_integrator = mm.CustomIntegrator(0.002*u.picoseconds) # 2 fs time step
custom_integrator.addComputePerDof('v', 'v+0.5*dt*f1/m') # OpenMM knows how to parse these strings
for _ in range(4):
    custom_integrator.addComputePerDof('v', 'v+0.5*(dt/4)*f0/m')
    custom_integrator.addComputePerDof('x', 'x+(dt/4)*v')
    custom_integrator.addComputePerDof('v', 'v+0.5*(dt/4)*f0/m')
custom_integrator.addComputePerDof('v', 'v+0.5*dt*f1/m')

print(custom_integrator)

<openmm.openmm.CustomIntegrator; proxy of <Swig Object of type 'OpenMM::CustomIntegrator *' at 0x7feb39dd3e10> >


## Simulation objects

The Simulation object creates a context that brings everything together.

In [12]:
simulation = app.Simulation(pdb.topology, system, integrator, platform)

Most of the useful simulation information is accessed through the 'context' class. This is the specific istantiation of your system as it is simulated. In fact, if you want, you can work directly with the context class as oppossed to the simulation class; however, the simulation class creates a context by default and bundles together reporter utilities in a nice way, so it is an excellent option.

In [13]:
print(simulation.context)

<openmm.openmm.Context; proxy of <Swig Object of type 'OpenMM::Context *' at 0x7feb39dd3870> >


The next level of abstraction from the context is the 'state'. This is more or less an in-memory report of the current *state* of the simulation. You can create a state from a context at any time; when you do this, you must specify what data (positions, velocities, energy, etc.) you want to access. Then you can get those quantities directly from the state object.

In [14]:
state = (simulation.context.getState(getPositions = True, getVelocities = True))
print(state)
print(state.getPositions()) # Note that currently all of the positions are 0.

<openmm.openmm.State; proxy of <Swig Object of type 'OpenMM::State *' at 0x7feb39dd3390> >
[Vec3(x=0.0, y=0.0, z=0.0), Vec3(x=0.0, y=0.0, z=0.0), Vec3(x=0.0, y=0.0, z=0.0), Vec3(x=0.0, y=0.0, z=0.0), Vec3(x=0.0, y=0.0, z=0.0), Vec3(x=0.0, y=0.0, z=0.0), Vec3(x=0.0, y=0.0, z=0.0), Vec3(x=0.0, y=0.0, z=0.0), Vec3(x=0.0, y=0.0, z=0.0), Vec3(x=0.0, y=0.0, z=0.0), Vec3(x=0.0, y=0.0, z=0.0), Vec3(x=0.0, y=0.0, z=0.0), Vec3(x=0.0, y=0.0, z=0.0), Vec3(x=0.0, y=0.0, z=0.0), Vec3(x=0.0, y=0.0, z=0.0), Vec3(x=0.0, y=0.0, z=0.0), Vec3(x=0.0, y=0.0, z=0.0), Vec3(x=0.0, y=0.0, z=0.0), Vec3(x=0.0, y=0.0, z=0.0), Vec3(x=0.0, y=0.0, z=0.0), Vec3(x=0.0, y=0.0, z=0.0), Vec3(x=0.0, y=0.0, z=0.0)] nm


You must explicitly set the positions and velocities of the particles after creating the simulation, which is easily done with the PDBFile object defined above and the .setVelocitiesToTemperature() method, respectively.

In [15]:
simulation.context.setPositions(pdb.positions)
simulation.context.setVelocitiesToTemperature(300*u.kelvin)
print(simulation.context.getState(getPositions=True).getPositions(asNumpy=True))
print(simulation.context.getState(getVelocities=True).getVelocities(asNumpy=True))

[[ 0.2     0.1    -0.    ]
 [ 0.2     0.209   0.    ]
 [ 0.1486  0.2454  0.089 ]
 [ 0.1486  0.2454 -0.089 ]
 [ 0.3427  0.2641 -0.    ]
 [ 0.4391  0.1877 -0.    ]
 [ 0.3555  0.397  -0.    ]
 [ 0.2733  0.4556 -0.    ]
 [ 0.4853  0.4614 -0.    ]
 [ 0.5408  0.4316  0.089 ]
 [ 0.5661  0.4221 -0.1232]
 [ 0.5123  0.4521 -0.2131]
 [ 0.663   0.4719 -0.1206]
 [ 0.5809  0.3141 -0.1241]
 [ 0.4713  0.6129  0.    ]
 [ 0.3601  0.6653  0.    ]
 [ 0.5846  0.6835  0.    ]
 [ 0.6737  0.6359 -0.    ]
 [ 0.5846  0.8284  0.    ]
 [ 0.4819  0.8648  0.    ]
 [ 0.636   0.8648  0.089 ]
 [ 0.636   0.8648 -0.089 ]] nm
[[ 2.40850459e+00  3.47706924e-01  7.45967432e-01]
 [ 1.73051733e-03  3.47706925e-01  1.17483804e-02]
 [ 1.52037474e+00 -3.57714501e-01  1.17731750e+00]
 [-1.02195568e+00 -2.56832962e+00 -5.89670278e-01]
 [-7.18912181e-01  2.35245202e-01 -2.12659845e-01]
 [-1.18072458e-01 -1.32253602e-01  8.51090996e-02]
 [-3.78384170e-01  4.12082551e-01 -7.90415554e-01]
 [-1.38521805e+00 -1.00023391e+00 -1.06918978

We can now run the simulation! We first minimize the energy of the system through a geometry relaxation, then we can step through our equations of motion. We'll only run a few steps here and save the state of the system. Once we have the tools to track our data, we'll let it go longer.

In [16]:
print("Minimizing energy...")
simulation.minimizeEnergy(maxIterations=100000,tolerance=0.000001)
print("Done minimizing energy.")
simulation.step(1)

Minimizing energy...
Done minimizing energy.


## More on the State object

The State object lets you pull information about the state of the system (positions, velocities, forces, energies, etc.). You must provide the specific information you want to be stored in the State, which can then be accessed by calling .getPositions(), .getVelocities(), etc. By default, these are returned as N Vec3 objects for N atoms in your system, but you can get Numpy arrays by setting asNumpy=True in the method argument.

In [17]:
current_state = simulation.context.getState(getPositions=True,
                                            getVelocities=True,
                                            getEnergy=True,
                                            getForces=True,
                                            getParameters=True,
                                            enforcePeriodicBox=True
                                            )
positions = current_state.getPositions()
velocities = current_state.getVelocities(asNumpy=True)
print(positions)
print(velocities)

[Vec3(x=0.19963185370353145, y=0.11820687177946378, z=10.05786317582109), Vec3(x=0.19846516362791244, y=0.21785838998473578, z=10.013712241677355), Vec3(x=0.135144703471111, y=0.28368919023626554, z=10.073192537955382), Vec3(x=0.16576502915693572, y=0.20867701978906508, z=9.910138486008524), Vec3(x=0.33882023228684777, y=0.269166679742621, z=10.021369336626158), Vec3(x=0.427871531908175, y=0.1991378201526536, z=10.071603123355054), Vec3(x=0.3613499594019822, y=0.3914035073744376, z=9.971481401853715), Vec3(x=0.28138706035440003, y=0.4411768764515504, z=9.935018138277583), Vec3(x=0.4919762765017514, y=0.459955097686348, z=9.973358508476652), Vec3(x=0.5464022872889448, y=0.4281604032533161, z=10.062284945684357), Vec3(x=0.5703869967919615, y=0.4136485619113789, z=9.85010338798742), Vec3(x=0.5175180020915123, y=0.44313079412566614, z=9.759457608692367), Vec3(x=0.6696785923928333, y=0.45846597218644874, z=9.846434523915528), Vec3(x=0.588283971424771, y=0.3061623412319968, z=9.8528123046053

## Saving states
We can save the State of our simulation as well by calling the saveState method on the simulation. There are actually two options to save at a given point in the simulation: saveState and saveCheckpoint. saveState creates a text file containing positions, velocities, forces, and many other parameters of the simulation. Because it is a text file, it can be used across different machines but can create a large document. Alternatively, saveCheckpoint saves a binary file with this information and should be only used on consistent hardware.

In [18]:
simulation.saveState('progress.state')
pos_check = simulation.context.getState(getPositions=True).getPositions(asNumpy=True) # check that loadState will give the same positions

## Reporters
Reporter objects store simulation data and save it to a file (.log, .pdb, .xtc, ...). You can choose how often you want to save a frame of the simulation (reportInterval), whether to have coordinates enforce periodicity (enforcePeriodicBox for PDBReporter), and what data you would like to keep. The data reporter must be appended to simulation.reporters to save any information about the simulation.

In addition to the one-off save options above, there is also a saveCheckpoint reporter that will automatically create a checkpoint file every $n$ steps in the simulation. There's an optional argument to save the system as a serialized State file (like .saveState) as well.

In [19]:
state_data_reporter = app.StateDataReporter(file='nvt_output.log',
                                            reportInterval=1,
                                            step=True,
                                            time=True,
                                            potentialEnergy=True,
                                            kineticEnergy=True,
                                            totalEnergy=True,
                                            temperature=True,
                                            volume=True,
                                            density=True,
                                            speed=True,
                                            separator=' ')

pdb_reporter = app.PDBReporter(file='nvt_output.pdb',
                               reportInterval=4,
                               enforcePeriodicBox=True)

checkpoint_reporter = app.CheckpointReporter(file='checkpnt.chk',
                                             reportInterval=5,
                                             writeState=False)

simulation.reporters.append(state_data_reporter)
simulation.reporters.append(pdb_reporter)
simulation.reporters.append(checkpoint_reporter)

TypeError: __init__() got an unexpected keyword argument 'writeState'

In [ ]:
simulation.step(10)

You should now see three new files in your directory: output.log, output.pdb, and checkpnt.chk.

## Loading states
Let's load the State we saved earlier. We'll first check that our positions have evolved and are not the same as those in the saved State.

In [ ]:
pos_check1 = simulation.context.getState(getPositions=True).getPositions(asNumpy=True) # check that positions are different than pos_check
print(np.all(pos_check == pos_check1)) # should be false

Now we can load the saved State and check if the positions are the same.

In [ ]:
simulation.loadState('progress.state')

pos_check2 = simulation.context.getState(getPositions=True).getPositions(asNumpy=True) # check that loadState will give the same positions
print(np.all(pos_check == pos_check2)) # should be true

## (Custom) Forces

We saw with thermostats and barostats that forces could be added outside of the forcefield rules. We can also add whichever forces we see fit using system.addForces(). A link to available forces is here: http://docs.openmm.org/development/api-python/library.html#forces. As we saw an example of adding pre-defined forces to the system above with thermostats and barostats, here we will focus on custom forces. OpenMM lets you write an analytical energy expression and apply the corresponding force to bonds, as an external force on particles, and more. Here we will write a force to trap atoms in our structure in an harmonic well.

There are different kinds of parameters we can set when defining a custom force. Global Parameters apply to all objects experiencing the force, whereas Per Particle Parameters are appropriately named and can have a unique value for every particle. In this case, our spring constant will be global and the positions in which we trap the atoms will be per particle.

In [ ]:
print(system.getForces()[-1]) # check the last force in the list, should change once we add custom force
custom_force = mm.CustomExternalForce('k*((x-x0)^2+(y-y0)^2+(z-z0)^2)') # energy function associated with the force we want to add
custom_force.addGlobalParameter('k', 1.0)
custom_force.addPerParticleParameter('x0')
custom_force.addPerParticleParameter('y0')
custom_force.addPerParticleParameter('z0');

Next, we want to add each particle to the force as well as the reference positions to each particle. We add particles according to their indices (enumerating over positions is a helpful way to get these), and we add the per particle parameter immediately after. Use the addParticle method on your defined force to do this.

In [ ]:
for i, position in enumerate(pos_check):
    custom_force.addParticle(i, position) # add particle to the force

We've created the force, but now we have to add it to the system.

In [ ]:
manual_remove_force(system, custom_force) # prevent adding a second custom force if cell rerun
system.addForce(custom_force)
print(system.getForces()[-1]) # check that the custom force is added

Although we've added the force to the system, we're not done yet. The context doesn't know that we've updated the system, so we must reinitialize it to have everyone on the same page. This is an expensive operation and gets rid of the State information of the system, so we have to be careful when calling it. We've already created a saved state, so let's not worry about creating a new checkpoint. Next, we call reinitialize on the context and reload the State information.

In [ ]:
# simulation.saveState('save_before_reinitialize.state')
simulation.context.reinitialize()
simulation.loadState('progress.state')

Let's try running.

In [ ]:
simulation.step(100)
print(system.getForces()[-1]) # check that the custom force is still there

Nice! It runs and the last force in our system is still the external force. Success!

We can also update the PerParticleParameters as we simulate. You may think, "Oh no! If I want to update the forces often, it'll be super expensive to save the State and reinitialize the Context each time." Fear not, for there is a quicker way. Feed the new perParticleParameters to your force object using the setParticleParameters method, then call updateParametersInContext to set them. Let's change the well from the pos_check positions to the pos_check1 positions.

Arguments (in order) for setParticleParameters:
- the index of the parameters you want to set for a given particle
- the index of the particle in your system for which you are setting parameters
- the parameters you would like to set

In [ ]:
# Check parameters of the custom force
particle_parameters = []
for i in range(custom_force.getNumParticles()):
    particle_parameters.append(custom_force.getParticleParameters(i))
print(particle_parameters)

# Change parameters of the custom force
for i, position in enumerate(pos_check1):
    custom_force.setParticleParameters(i, i, (position[0], position[1], position[2]))
custom_force.updateParametersInContext(simulation.context)

# Check parameters of the custom force again
particle_parameters = []
for i in range(custom_force.getNumParticles()):
    particle_parameters.append(custom_force.getParticleParameters(i))
print(particle_parameters)

The force parameters have changed!